In [ ]:
%pip install langchain langchain-community langchain-text-splitters

In [ ]:
%pip install langchain-openai

In [ ]:
%pip install langchain_pinecone

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

with open("./analects_of_confucius.txt", "r", encoding="utf-8") as f:
    text = f.read()

docs = [Document(page_content=text)]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 문서를 쪼갰을 때 하나의 청크가 가질 수 있는 토큰 수
    chunk_overlap=200,
)

document_list = text_splitter.split_documents(docs)

In [ ]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()

# Upstage에서 제공하는 Embedding Model을 활용해서 chunk를 vector화
embedding=UpstageEmbeddings(model="solar-embedding-1-large")

In [ ]:
from langchain_pinecone import PineconeVectorStore
index_name = 'analects-upstage-index'

# Document ID를 지정한 게 아니라면 실행할 때마다 중복된 문서들이 들어갈 수 있다.
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

# 한번 저장 후 나중에는 이 코드 활용
# database = PineconeVectorStore.from_existing_index(
#     index_name=index_name,
#     embedding=embedding
# )

In [ ]:
query = '공부 태도를 제대로 하고 싶어.'

# retrieved_docs = database.similarity_search(query, k=3)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [ ]:
from langchain_classic import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from langchain_classic.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs = {"prompt":prompt}
)

In [ ]:
ai_message = qa_chain({"query": query})

In [ ]:
ai_message

In [ ]:
ai_message2 = qa_chain.invoke({"query": "선생님으로서 학급을 잘 운영하고싶어."})

In [ ]:
ai_message2